In [ ]:
import polars as pl
import duckdb
import plotly.graph_objs as go
import plotly.express as px
import missingno as msno

In [ ]:
EUROAREA = [
    "Austria",
    "Belgium",
    "Cyprus",
    "Estonia",
    "Finland",
    "France and Monaco",
    "Germany",
    "Greece",
    "Ireland",
    "Italy, San Marino and the Holy See",
    "Latvia",
    "Lithuania",
    "Luxembourg",
    "Malta",
    "Netherlands",
    "Portugal",
    "Slovakia",
    "Slovenia",
    "Spain and Andorra",
]

# Reading Dataset and Initial Exploration

In [ ]:
df = pl.read_csv("../data/GHG_total_by_country.csv")
df.head()

In [ ]:
# Summary of statistics
df.describe()

In [ ]:
# Check for missing values
with pl.Config() as cfg:
    cfg.set_tbl_cols(-1)
    print(df.null_count())

msno.matrix(df.to_pandas(), sparkline=False)

check_nulls = pl.any_horizontal(pl.col(col).is_null() for col in df.columns)
rows_with_nulls = df.filter(check_nulls)
with pl.Config() as cfg:
    cfg.set_tbl_cols(-1)
    print(rows_with_nulls)

# Data Cleaning and Preprocessing

In [ ]:
# DuckDB connection
con = duckdb.connect()
con.register("df_table", df)

# ETL data for Euro area
euroarea_countries = ", ".join(f"'{country}'" for country in EUROAREA)
df_euroarea = con.execute(
    f"""
    SELECT * FROM df_table WHERE Country IN ({euroarea_countries})
"""
).pl()
df_euroarea = df_euroarea.mean()
df_euroarea = df_euroarea.with_columns(
    pl.col("EDGAR Country Code").fill_null("EUROAREA"),
    pl.col("Country").fill_null("EUROAREA"),
)
df_euroarea = df_euroarea.with_columns(
    [
        pl.col(col).round(5)
        for col in df_euroarea.columns
        if df_euroarea[col].dtype in (pl.Float64, pl.Float32)
    ]
)

# ETL data for European Union (EU27)
df_eu27 = con.execute(
    """
    SELECT * FROM df_table WHERE "EDGAR Country Code" = 'EU27' AND "Country" = 'EU27'
"""
).pl()

con.close()

# ETL data for worldwide
df_worldwide = df.filter(
    (pl.col("EDGAR Country Code") == "GLOBAL TOTAL")
    & (pl.col("Country") == "GLOBAL TOTAL")
)
df_worldwide = df_worldwide.with_columns(
    pl.col("EDGAR Country Code").replace("GLOBAL TOTAL", "WORLDWIDE"),
    pl.col("Country").replace("GLOBAL TOTAL", "WORLDWIDE"),
)

# Visualisation

In [ ]:
with pl.Config() as cfg:
    cfg.set_tbl_cols(-1)
    cfg.set_tbl_rows(-1)
    print(df_euroarea.to_pandas().to_csv())
    print()
    print(df_eu27.to_pandas().to_csv())
    print()
    print(df_worldwide.to_pandas().to_csv())

In [ ]:
years = df_euroarea.columns[2:]
values1 = df_euroarea.row(0)[2:]
values2 = df_eu27.row(0)[2:]
values3 = df_worldwide.row(0)[2:]

# Create traces
trace1 = go.Scatter(x=years, y=values1, mode="lines", name="EUROAREA")
trace2 = go.Scatter(x=years, y=values2, mode="lines", name="EU27")
trace3 = go.Scatter(x=years, y=values3, mode="lines", name="WORLDWIDE")

# Create the figure
fig = go.Figure(data=[trace1, trace2, trace3])

# Update layout
fig.update_layout(
    title="GHG Growth Evolution",
    xaxis_title="Year",
    yaxis_title="CO2eq (Mt)",
    xaxis=dict(tickmode="linear", tick0=1970, dtick=5),
    template="plotly_white",
)

fig.show()